# Optional Lab - Softmax Function
In this lab, we will explore the softmax function. This function is used in both Softmax Regression and in Neural Networks when solving Multiclass Classification problems.  

<center>  <img  src="./images/C2_W2_Softmax_Header.PNG" width="600" />  <center/>

  

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from IPython.display import display, Markdown, Latex
from sklearn.datasets import make_blobs
%matplotlib widget
from matplotlib.widgets import Slider
from lab_utils_common import dlc
from lab_utils_softmax import plt_softmax
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

> **Note**: Normally, in this course, the notebooks use the convention of starting counts with 0 and ending with N-1,  $\sum_{i=0}^{N-1}$, while lectures start with 1 and end with N,  $\sum_{i=1}^{N}$. This is because code will typically start iteration with 0 while in lecture, counting 1 to N leads to cleaner, more succinct equations. This notebook has more equations than is typical for a lab and thus  will break with the convention and will count 1 to N.

## Softmax Function
In both softmax regression and neural networks with Softmax outputs, N outputs are generated and one output is selected as the predicted category. In both cases a vector $\mathbf{z}$ is generated by a linear function which is applied to a softmax function. The softmax function converts $\mathbf{z}$  into a probability distribution as described below. After applying softmax, each output will be between 0 and 1 and the outputs will add to 1, so that they can be interpreted as probabilities. The larger inputs  will correspond to larger output probabilities.
<center>  <img  src="./images/C2_W2_SoftmaxReg_NN.png" width="600" />  

The softmax function can be written:
$$a_j = \frac{e^{z_j}}{ \sum_{k=1}^{N}{e^{z_k} }} \tag{1}$$
The output $\mathbf{a}$ is a vector of length N, so for softmax regression, you could also write:
\begin{align}
\mathbf{a}(x) =
\begin{bmatrix}
P(y = 1 | \mathbf{x}; \mathbf{w},b) \\
\vdots \\
P(y = N | \mathbf{x}; \mathbf{w},b)
\end{bmatrix}
=
\frac{1}{ \sum_{k=1}^{N}{e^{z_k} }}
\begin{bmatrix}
e^{z_1} \\
\vdots \\
e^{z_{N}} \\
\end{bmatrix} \tag{2}
\end{align}


Which shows the output is a vector of probabilities. The first entry is the probability the input is the first category given the input $\mathbf{x}$ and parameters $\mathbf{w}$ and $\mathbf{b}$.  
Let's create a NumPy implementation:

In [2]:
def my_softmax(z):
    ez = np.exp(z)              #element-wise exponenial
    sm = ez/np.sum(ez)
    return(sm)

Below, vary the values of the `z` inputs using the sliders.

In [3]:
plt.close("all")
plt_softmax(my_softmax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

As you are varying the values of the z's above, there are a few things to note:
* the exponential in the numerator of the softmax magnifies small differences in the values 
* the output values sum to one
* the softmax spans all of the outputs. A change in `z0` for example will change the values of `a0`-`a3`. Compare this to other activations such as ReLu or Sigmoid which have a single input and single output.

## Cost
<center> <img  src="./images/C2_W2_SoftMaxCost.png" width="400" />    <center/>

The loss function associated with Softmax, the cross-entropy loss, is:
\begin{equation}
  L(\mathbf{a},y)=\begin{cases}
    -log(a_1), & \text{if $y=1$}.\\
        &\vdots\\
     -log(a_N), & \text{if $y=N$}
  \end{cases} \tag{3}
\end{equation}

Where y is the target category for this example and $\mathbf{a}$ is the output of a softmax function. In particular, the values in $\mathbf{a}$ are probabilities that sum to one.
>**Recall:** In this course, Loss is for one example while Cost covers all examples. 
 
 
Note in (3) above, only the line that corresponds to the target contributes to the loss, other lines are zero. To write the cost equation we need an 'indicator function' that will be 1 when the index matches the target and zero otherwise. 
    $$\mathbf{1}\{y == n\} = =\begin{cases}
    1, & \text{if $y==n$}.\\
    0, & \text{otherwise}.
  \end{cases}$$
Now the cost is:
\begin{align}
J(\mathbf{w},b) = - \left[ \sum_{i=1}^{m} \sum_{j=1}^{N}  1\left\{y^{(i)} == j\right\} \log \frac{e^{z^{(i)}_j}}{\sum_{k=1}^N e^{z^{(i)}_k} }\right] \tag{4}
\end{align}

Where $m$ is the number of examples, $N$ is the number of outputs. This is the average of all the losses.


## Tensorflow
This lab will discuss two ways of implementing the softmax, cross-entropy loss in Tensorflow, the 'obvious' method and the 'preferred' method. The former is the most straightforward while the latter is more numerically stable.

Let's start by creating a dataset to train a multiclass classification model.

In [4]:
# make  dataset for example
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
X_train, y_train = make_blobs(n_samples=2000, centers=centers, cluster_std=1.0,random_state=30)

### The *Obvious* organization

The model below is implemented with the softmax as an activation in the final Dense layer.
The loss function is separately specified in the `compile` directive. 

The loss function `SparseCategoricalCrossentropy`. The loss described in (3) above. In this model, the softmax takes place in the last layer. The loss function takes in the softmax output which is a vector of probabilities. 

In [5]:
model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'softmax')    # < softmax activation here
    ]
)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    X_train,y_train,
    epochs=10
)
        

Epoch 1/10
63/63 [==============================] - 0s 958us/step - loss: 0.8390
Epoch 2/10
63/63 [==============================] - 0s 1ms/step - loss: 0.3807
Epoch 3/10
63/63 [==============================] - 0s 1ms/step - loss: 0.1923
Epoch 4/10
63/63 [==============================] - 0s 1ms/step - loss: 0.1094
Epoch 5/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0751
Epoch 6/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0602
Epoch 7/10
63/63 [==============================] - 0s 950us/step - loss: 0.0510
Epoch 8/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0449
Epoch 9/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0408
Epoch 10/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0377


Because the softmax is integrated into the output layer, the output is a vector of probabilities.

In [6]:
p_nonpreferred = model.predict(X_train)
print(p_nonpreferred [:2])
print("largest value", np.max(p_nonpreferred), "smallest value", np.min(p_nonpreferred))

[[1.66e-03 1.52e-03 9.85e-01 1.16e-02]
 [9.93e-01 6.35e-03 4.36e-04 8.06e-06]]
largest value 0.99999964 smallest value 2.3815372e-11


### Preferred <img align="Right" src="./images/C2_W2_softmax_accurate.png"  style=" width:400px; padding: 10px 20px ; ">
Recall from lecture, more stable and accurate results can be obtained if the softmax and loss are combined during training.   This is enabled by the 'preferred' organization shown here.


In the preferred organization the final layer has a linear activation. For historical reasons, the outputs in this form are referred to as *logits*. The loss function has an additional argument: `from_logits = True`. This informs the loss function that the softmax operation should be included in the loss calculation. This allows for an optimized implementation.

In [7]:
preferred_model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'linear')   #<-- Note
    ]
)
preferred_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  #<-- Note
    optimizer=tf.keras.optimizers.Adam(0.001),
)

preferred_model.fit(
    X_train,y_train,
    epochs=10
)
        

Epoch 1/10
63/63 [==============================] - 0s 949us/step - loss: 1.0512
Epoch 2/10
63/63 [==============================] - 0s 937us/step - loss: 0.3729
Epoch 3/10
63/63 [==============================] - 0s 1ms/step - loss: 0.1545
Epoch 4/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0956
Epoch 5/10
63/63 [==============================] - 0s 923us/step - loss: 0.0725
Epoch 6/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0607
Epoch 7/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0531
Epoch 8/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0475
Epoch 9/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0431
Epoch 10/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0396


#### Output Handling
Notice that in the preferred model, the outputs are not probabilities, but can range from large negative numbers to large positive numbers. The output must be sent through a softmax when performing a prediction that expects a probability. 
Let's look at the preferred model outputs:

In [8]:
p_preferred = preferred_model.predict(X_train)
print(f"two example output vectors:\n {p_preferred[:2]}")
print("largest value", np.max(p_preferred), "smallest value", np.min(p_preferred))

two example output vectors:
 [[-2.94 -2.33  2.86 -1.25]
 [ 1.5  -4.28 -7.08 -7.93]]
largest value 8.857447 smallest value -13.404879


The output predictions are not probabilities!
If the desired output are probabilities, the output should be be processed by a [softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax).

In [9]:
sm_preferred = tf.nn.softmax(p_preferred).numpy()
print(f"two example output vectors:\n {sm_preferred[:2]}")
print("largest value", np.max(sm_preferred), "smallest value", np.min(sm_preferred))

two example output vectors:
 [[2.97e-03 5.46e-03 9.75e-01 1.62e-02]
 [9.97e-01 3.08e-03 1.86e-04 8.00e-05]]
largest value 0.99999774 smallest value 1.0387312e-07


To select the most likely category, the softmax is not required. One can find the index of the largest output using [np.argmax()](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html).

In [10]:
for i in range(5):
    print( f"{p_preferred[i]}, category: {np.argmax(p_preferred[i])}")

[-2.94 -2.33  2.86 -1.25], category: 2
[ 1.5  -4.28 -7.08 -7.93], category: 0
[ 1.02 -2.93 -5.43 -6.26], category: 0
[-2.19  3.48 -1.81 -2.91], category: 1
[-2.32 -6.31  3.67 -4.91], category: 2


## SparseCategorialCrossentropy or CategoricalCrossEntropy
Tensorflow has two potential formats for target values and the selection of the loss defines which is expected.
- SparseCategorialCrossentropy: expects the target to be an integer corresponding to the index. For example, if there are 10 potential target values, y would be between 0 and 9. 
- CategoricalCrossEntropy: Expects the target value of an example to be one-hot encoded where the value at the target index is 1 while the other N-1 entries are zero. An example with 10 potential target values, where the target is 2 would be [0,0,1,0,0,0,0,0,0,0].


## Congratulations!
In this lab you 
- Became more familiar with the softmax function and its use in softmax regression and in softmax activations in neural networks. 
- Learned the preferred model construction in Tensorflow:
    - No activation on the final layer (same as linear activation)
    - SparseCategoricalCrossentropy loss function
    - use from_logits=True
- Recognized that unlike ReLu and Sigmoid, the softmax spans multiple outputs.

## Numerical Stability (optional)
This section discusses some of the methods employed to improve numerical stability. This is for the interested reader and is not at all required.

### Softmax Numerical Stability
The input's  to the softmax are the outputs of a linear layer $z_j = \mathbf{w_j} \cdot \mathbf{x}^{(i)}+b$. These may
be large numbers. The first step of the softmax algorithm computes $e^{z_j}$. This can result in an overflow error if the number gets too large. Try running the cell below:

In [11]:
for z in [500,600,700,800]:
    ez = np.exp(z)
    zs = "{" + f"{z}" + "}"
    print(f"e^{zs} = {ez:0.2e}")

e^{500} = 1.40e+217
e^{600} = 3.77e+260
e^{700} = 1.01e+304
e^{800} = inf


The operation will generate an overflow if the exponent gets too large. Naturally, `my_softmax()` will generate the same errors:

In [12]:
z_tmp = np.array([[500,600,700,800]])
my_softmax(z_tmp)

array([[ 0.,  0.,  0., nan]])

Numerical stability can be improved by reducing the size of the exponent. 
Recall 
$$ e^{a + b} = e^ae^b$$
if the $b$  were the opposite sign of $a$, this would reduce the size of the exponent. Specifically, if you multiplied the softmax by a fraction:
$$a_j = \frac{e^{z_j}}{ \sum_{i=1}^{N}{e^{z_i} }} \frac{e^{-b}}{ {e^{-b}}}$$
the exponent would be reduced and the value of the softmax would not change. If $b$ in $e^b$ were the largest value of the $z_j$'s, $max_j(\mathbf{z})$, the exponent would be reduced to its smallest value.
$$\begin{align}
a_j &= \frac{e^{z_j}}{ \sum_{i=1}^{N}{e^{z_i} }} \frac{e^{-max_j(\mathbf{z})}}{ {e^{-max_j(\mathbf{z})}}} \\
&= \frac{e^{z_j-max_j(\mathbf{z})}}{ \sum_{i=1}^{N}{e^{z_i-max_j(\mathbf{z})} }} 
\end{align}$$
It is customary to say $C=max_j(\mathbf{z})$ since the equation would be correct with any constant C. 

$$
a_j = \frac{e^{z_j-C}}{ \sum_{i=1}^{N}{e^{z_i-C} }} \quad\quad\text{where}\quad C=max_j(\mathbf{z})\tag{5}
$$

If we look at our troublesome example where $\mathbf{z}$ contains 500,600,700,800, $C=max_j(\mathbf{z})=800$:
\begin{align}
\mathbf{a}(x) =
\frac{1}{ e^{500-800} + e^{600-800} + e^{700-800} + e^{800-800}}
\begin{bmatrix}
e^{500-800} \\
e^{600-800} \\
e^{700-800} \\
e^{800-800} \\
\end{bmatrix}
= 
\begin{bmatrix}
5.15e-131 \\
1.38e-87 \\
3.7e-44 \\
1.0 \\
\end{bmatrix}
\end{align}

Let's rewrite `my_softmax` to improve its numerical stability.

In [18]:
def my_softmax_ns(z):
    """numerically stablility improved"""
    bigz = np.max(z)
    ez = np.exp(z-bigz)              # minimize exponent
    sm = ez/np.sum(ez)
    return(sm)

Let's try this and compare it to the tensorflow implementation:

In [20]:
z_tmp = np.array([500.,600,700,800])
print(tf.nn.softmax(z_tmp).numpy(), "\n", my_softmax_ns(z_tmp))

[5.15e-131 1.38e-087 3.72e-044 1.00e+000] 
 [5.15e-131 1.38e-087 3.72e-044 1.00e+000]


Large values no longer cause an overflow.

### Cross Entropy Loss Numerical Stability

The loss function associated with Softmax, the cross-entropy loss, is repeated here:
\begin{equation}
  L(\mathbf{a},y)=\begin{cases}
    -log(a_1), & \text{if $y=1$}.\\
        &\vdots\\
     -log(a_N), & \text{if $y=N$}
  \end{cases} 
\end{equation}

Where y is the target category for this example and $\mathbf{a}$ is the output of a softmax function. In particular, the values in $\mathbf{a}$ are probabilities that sum to one.
Let's consider a case where the target is two ($y=2$) and just look at the loss for that case. This will result in the loss being:  
$$L(\mathbf{a})= -log(a_2)$$
Recall that $a_2$ is the output of the softmax function described above, so this can be written:
$$L(\mathbf{z})= -log\left(\frac{e^{z_2}}{ \sum_{i=1}^{N}{e^{z_i} }}\right) \tag{6}$$
This can be optimized. However, to make those optimizations, the softmax and the loss must be calculated together as shown in the 'preferred' Tensorflow implementation you saw above.

Starting from (6) above, the loss for the case of y=2:
$log(\frac{a}{b}) = log(a) - log(b)$, so (6) can be rewritten:
$$L(\mathbf{z})= -\left[log(e^{z_2}) - log \sum_{i=1}^{N}{e^{z_i} }\right] \tag{7}$$
The first term can be simplified to just $z_2$:
$$L(\mathbf{z})= -\left[z_2 - log( \sum_{i=1}^{N}{e^{z_i} })\right] =  \underbrace{log \sum_{i=1}^{N}{e^{z_i} }}_\text{logsumexp()} -z_2 \tag{8}$$
It turns out that the $log \sum_{i=1}^{N}{e^{z_i} }$ term in the above equation is so often used, many libraries have an implementation. In Tensorflow this is tf.math.reduce_logsumexp(). An issue with this sum is that the exponent in the sum could overflow if $z_i$ is large. To fix this, we might like to subtract $e^{max_j(\mathbf{z})}$ as we did above, but this will require a bit of work:
$$
\begin{align}
   log \sum_{i=1}^{N}{e^{z_i} } &= log \sum_{i=1}^{N}{e^{(z_i - max_j(\mathbf{z}) + max_j(\mathbf{z}))}} \tag{9}\\
                          &= log \sum_{i=1}^{N}{e^{(z_i - max_j(\mathbf{z}))} e^{max_j(\mathbf{z})}} \\
                          &= log(e^{max_j(\mathbf{z})}) + log \sum_{i=1}^{N}{e^{(z_i - max_j(\mathbf{z}))}} \\
                          &= max_j(\mathbf{z})  + log \sum_{i=1}^{N}{e^{(z_i - max_j(\mathbf{z}))}}
\end{align}
$$
Now, the exponential is less likely to overflow. It is customary to say $C=max_j(\mathbf{z})$ since the equation would be correct with any constant C. We can now write the loss equation:
    
$$L(\mathbf{z})= C+ log( \sum_{i=1}^{N}{e^{z_i-C} }) -z_2  \;\;\;\text{where } C=max_j(\mathbf{z}) \tag{10} $$
A computationally simpler, more stable version of the loss. The above is for an example where the target, y=2 but generalizes to any target.